# Understanding the Transformer Model from scratch

The Transformer model is a groundbreaking architecture in AI, especially for Natural Language Processing (NLP) tasks like language translation, text generation, and more. It was introduced in 2017 by Vaswani et al. in a paper titled "Attention Is All You Need". This model was designed to handle sequential data (like sentences) in a much more efficient way than previous models like Recurrent Neural Networks (RNNs) and Long Short-Term Memory (LSTM) networks.

## Key Components of the Transformer

The transformer architecture has a few critical building blocks that set it apart:

### a. Self-Attention Mechanism

What is Attention? The self-attention mechanism helps the model figure out which words in a sentence are most important relative to other words. For example, in the sentence "The cat sat on the mat.", the model needs to understand that "sat" relates closely to both "cat" and "mat."

Why Self-Attention? Traditional RNNs processed words one-by-one (sequentially), which makes it harder to understand long-range dependencies. With self-attention, the model can look at all words in the sentence at the same time and weigh their relationships.

How Does It Work? Self-attention takes in three components for each word:
- Query (Q)
- Key (K)
- Value (V)

Each word in a sentence will compute attention scores by comparing its Query to all other words' Keys. These attention scores tell us how much focus should be placed on each word when generating a word’s output. The result is weighted by the Value of the word.

### b. Multi-Head Attention

Instead of having just one attention mechanism, the transformer model uses multiple attention heads. Each head learns a different aspect of the relationships between words in the sentence.

Why Multiple Heads? With multiple heads, the model can capture different patterns in the relationships. For instance, one head may focus on subject-verb relationships, while another might focus on noun-adjective pairings.

### c. Positional Encoding

Transformers process input data all at once (in parallel), so they don't inherently know the order of the words in a sentence.

To fix this, positional encoding is added to the input embeddings to represent the position of each word in the sequence. This encoding ensures the model understands the order of the words, which is crucial for meaning.

### d. Feed-Forward Neural Networks

After the attention layers, the transformer uses feed-forward layers (fully connected layers) to process the output from the attention mechanism and transform the data further.

These layers help learn complex transformations that improve the overall model’s performance.

## Why Transformers Are So Effective
### a. Parallelization

One of the key advantages of transformers is that they process all the words in a sentence simultaneously (in parallel). Unlike RNNs or LSTMs, which process one word at a time, transformers don’t have to wait for the previous word’s processing to be completed before moving on to the next one.
This makes transformers much faster to train and scalable to large datasets.

### b. Long-Range Dependencies

Transformers are better at capturing long-range dependencies in text. For instance, in a sentence like “The man who was standing at the door left the house,” the model can connect “man” with “left” even though there are words in between. Traditional models like RNNs struggle with such dependencies, especially when sentences are long.

### c. Flexibility in Tasks

Transformers can handle a variety of tasks like translation, summarization, question-answering, and text generation.
This flexibility is because the encoder-decoder architecture can be adapted. For instance, the decoder part can be used for generating text, while the encoder can be used for understanding and classifying text.

## How Transformers are Used in Real Life

Transformers have revolutionized the field of AI, and they're the backbone of many cutting-edge models:

- GPT (Generative Pre-trained Transformer): This is the model behind AI tools like ChatGPT. It uses a transformer decoder architecture to generate human-like text.
- BERT (Bidirectional Encoder Representations from Transformers): BERT uses the transformer encoder to understand context and perform tasks like sentiment analysis, named entity recognition, and more.
- T5 (Text-to-Text Transfer Transformer): T5 treats every NLP problem as a text-to-text problem. For example, translating text from one language to another is treated as "Translate English to French: [text]."

In [74]:
import torch
import torch.nn as nn
import math
from torch.utils.data import Dataset
from pathlib import Path

In [75]:
class InputEmbeddings(nn.Module):
    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)
    
    def forward(self, x):
        return self.embedding(x) * math.sqrt(self.d_model)

In [76]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, seq: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.seq = seq
        self.dropout = nn.Dropout(dropout)
        
        # Create a matrix of shape (seq, d_model)
        pe = torch.zeros(seq, d_model)
        
        # Create a vector of shape (seq)
        position = torch.arange(0, seq, dtype=torch.float).unsqueeze(1) # (seq, 1)
        
        # Create a vector of shape (d_model)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # (d_model / 2)
        
        # Apply sine to even indices
        pe[:, 0::2] = torch.sin(position * div_term) # sin(position * (10000 ** (2i / d_model))
        
        # Apply cosine to odd indices
        pe[:, 1::2] = torch.cos(position * div_term) # cos(position * (10000 ** (2i / d_model))
        
        # Add a batch dimension to the positional encoding
        pe = pe.unsqueeze(0) # (1, seq, d_model)
        
        # Register the positional encoding as a buffer
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False) # (batch, seq, d_model)
        return self.dropout(x)

In [77]:
class LayerNormalization(nn.Module):

    def __init__(self, features: int, eps:float=10**-6) -> None:
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(features)) # alpha (multiplicative) is a learnable parameter
        self.bias = nn.Parameter(torch.zeros(features)) # bias (additive) is a learnable parameter

    def forward(self, x):
        # x: (batch, seq, hidden_size)
        # Keep the dimension for broadcasting
        mean = x.mean(dim = -1, keepdim = True) # (batch, seq, 1)
        # Keep the dimension for broadcasting
        std = x.std(dim = -1, keepdim = True) # (batch, seq, 1)
        # eps is to prevent dividing by zero or when std is very small
        return self.alpha * (x - mean) / (std + self.eps) + self.bias

In [78]:
class FeedForwardBlock(nn.Module):

    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff) # w1 and b1
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model) # w2 and b2

    def forward(self, x):
        # (batch, seq, d_model) --> (batch, seq, d_ff) --> (batch, seq, d_model)
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

In [61]:
class MultiHeadAttentionBlock(nn.Module):

    def __init__(self, d_model: int, h: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model # Embedding vector size
        self.h = h # Number of heads
        # Make sure d_model is divisible by h
        assert d_model % h == 0, "d_model is not divisible by h"

        self.d_k = d_model // h # Dimension of vector seen by each head
        self.w_q = nn.Linear(d_model, d_model, bias=False) # Wq
        self.w_k = nn.Linear(d_model, d_model, bias=False) # Wk
        self.w_v = nn.Linear(d_model, d_model, bias=False) # Wv
        self.w_o = nn.Linear(d_model, d_model, bias=False) # Wo
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        d_k = query.shape[-1]
        # Just apply the formula from the paper
        # (batch, h, seq, d_k) --> (batch, h, seq, seq)
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            # Write a very low value (indicating -inf) to the positions where mask == 0
            attention_scores.masked_fill_(mask == 0, -1e9)
        attention_scores = attention_scores.softmax(dim=-1) # (batch, h, seq, seq) # Apply softmax
        if dropout is not None:
            attention_scores = dropout(attention_scores)
        # (batch, h, seq, seq) --> (batch, h, seq, d_k)
        # return attention scores which can be used for visualization
        return (attention_scores @ value), attention_scores

    def forward(self, q, k, v, mask):
        query = self.w_q(q) # (batch, seq, d_model) --> (batch, seq, d_model)
        key = self.w_k(k) # (batch, seq, d_model) --> (batch, seq, d_model)
        value = self.w_v(v) # (batch, seq, d_model) --> (batch, seq, d_model)

        # (batch, seq, d_model) --> (batch, seq, h, d_k) --> (batch, h, seq, d_k)
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

        # Calculate attention
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)
        
        # Combine all the heads together
        # (batch, h, seq, d_k) --> (batch, seq, h, d_k) --> (batch, seq, d_model)
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

        # Multiply by Wo
        # (batch, seq, d_model) --> (batch, seq, d_model)  
        return self.w_o(x)

In [62]:
class ResidualConnection(nn.Module):
    
        def __init__(self, features: int, dropout: float) -> None:
            super().__init__()
            self.dropout = nn.Dropout(dropout)
            self.norm = LayerNormalization(features)
    
        def forward(self, x, sublayer):
            return x + self.dropout(sublayer(self.norm(x)))

## Transformer Architecture: Encoder-Decoder Structure

The transformer model is typically divided into two parts:

- Encoder: This part processes the input data (e.g., a sentence in one language).
- Decoder: This part generates the output data (e.g., the translation of that sentence).

Each of these parts is made up of layers that are stacked on top of each other:
- Encoder Layers: Each encoder layer consists of two main components:
    - Multi-head self-attention
    - Feed-forward neural network
- Decoder Layers: The decoder is similar, but with an additional layer of masked multi-head self-attention (which ensures that the decoder cannot "cheat" by looking ahead at future words).

In [63]:
class EncoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(2)])

    def forward(self, x, src_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x

In [64]:
class Encoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [65]:
class DecoderBlock(nn.Module):

    def __init__(
        self, 
        features: int, 
        self_attention_block: MultiHeadAttentionBlock, 
        cross_attention_block: MultiHeadAttentionBlock, 
        feed_forward_block: FeedForwardBlock, 
        dropout: float
    ) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(3)])

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x

In [66]:
class Decoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return self.norm(x)

In [67]:
class ProjectionLayer(nn.Module):

    def __init__(self, d_model, vocab_size) -> None:
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x) -> None:
        # (batch, seq, d_model) --> (batch, seq, vocab_size)
        return self.proj(x)

In [68]:
class Transformer(nn.Module):

    def __init__(
        self, 
        encoder: Encoder, 
        decoder: Decoder, 
        src_embed: InputEmbeddings, 
        tgt_embed: InputEmbeddings, 
        src_pos: PositionalEncoding, 
        tgt_pos: PositionalEncoding, 
        projection_layer: ProjectionLayer
    ) -> None:
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer

    def encode(self, src, src_mask):
        # (batch, seq, d_model)
        src = self.src_embed(src)
        src = self.src_pos(src)
        return self.encoder(src, src_mask)
    
    def decode(self, encoder_output: torch.Tensor, src_mask: torch.Tensor, tgt: torch.Tensor, tgt_mask: torch.Tensor):
        # (batch, seq, d_model)
        tgt = self.tgt_embed(tgt)
        tgt = self.tgt_pos(tgt)
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)
    
    def project(self, x):
        # (batch, seq, vocab_size)
        return self.projection_layer(x)

In [69]:
def build_transformer(
    src_vocab_size: int, 
    tgt_vocab_size: int, 
    src_seq: int, 
    tgt_seq: int, 
    d_model: int=512, 
    N: int=6, 
    h: int=8, 
    dropout: float=0.1, 
    d_ff: int=2048
) -> Transformer:
    # Create the embedding layers
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

    # Create the positional encoding layers
    src_pos = PositionalEncoding(d_model, src_seq, dropout)
    tgt_pos = PositionalEncoding(d_model, tgt_seq, dropout)
    
    # Create the encoder blocks
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        encoder_block = EncoderBlock(d_model, encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)

    # Create the decoder blocks
    decoder_blocks = []
    for _ in range(N):
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        decoder_block = DecoderBlock(d_model, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)
    
    # Create the encoder and decoder
    encoder = Encoder(d_model, nn.ModuleList(encoder_blocks))
    decoder = Decoder(d_model, nn.ModuleList(decoder_blocks))
    
    # Create the projection layer
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)
    
    # Create the transformer
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)
    
    # Initialize the parameters
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    
    return transformer

[SOS] → Signals the start of a sentence (important for decoding).
[EOS] → Indicates the end of a sentence (used in labels).
[PAD] → Pads shorter sentences to match seq_len (ignored during computation).

## 

In [70]:
class BPETokenizer:
    def __init__(self, num_merges=100):
        self.num_merges = num_merges
        self.vocab = {}
        self.merges = {}

        # Special tokens
        self.special_tokens = {
            "[SOS]": 0,
            "[EOS]": 1,
            "[PAD]": 2
        }

    def train(self, corpus):
        """
        Train the BPE tokenizer on a given text corpus.
        """
        word_freqs = {}
        for sentence in corpus:
            words = sentence.split()
            for word in words:
                word_freqs[word] = word_freqs.get(word, 0) + 1

        # Initialize vocabulary with characters
        self.vocab = {char: idx + 3 for idx, char in enumerate(set("".join(word_freqs.keys())))}
        self.vocab.update(self.special_tokens)

        # Merge operations
        for _ in range(self.num_merges):
            pairs = {}
            for word, freq in word_freqs.items():
                symbols = list(word)
                for i in range(len(symbols) - 1):
                    pair = (symbols[i], symbols[i + 1])
                    pairs[pair] = pairs.get(pair, 0) + freq

            if not pairs:
                break

            best_pair = max(pairs, key=pairs.get)
            new_symbol = "".join(best_pair)
            self.vocab[new_symbol] = len(self.vocab)
            self.merges[best_pair] = new_symbol

            new_word_freqs = {}
            for word, freq in word_freqs.items():
                symbols = list(word)
                i = 0
                while i < len(symbols) - 1:
                    pair = (symbols[i], symbols[i + 1])
                    if pair == best_pair:
                        symbols[i] = new_symbol
                        del symbols[i + 1]
                    i += 1
                new_word_freqs["".join(symbols)] = freq
            word_freqs = new_word_freqs

    def encode(self, text):
        """
        Encode text into token IDs.
        """
        symbols = list(text)
        while len(symbols) > 1:
            pairs = [(symbols[i], symbols[i + 1]) for i in range(len(symbols) - 1)]
            best_pair = None
            for pair in pairs:
                if pair in self.merges:
                    best_pair = pair
                    break
            if best_pair is None:
                break
            new_symbol = self.merges[best_pair]
            new_symbols = []
            i = 0
            while i < len(symbols):
                if i < len(symbols) - 1 and (symbols[i], symbols[i + 1]) == best_pair:
                    new_symbols.append(new_symbol)
                    i += 2
                else:
                    new_symbols.append(symbols[i])
                    i += 1
            symbols = new_symbols

        return [self.special_tokens["[SOS]"]] + [self.vocab[s] for s in symbols if s in self.vocab] + [self.special_tokens["[EOS]"]]

    def decode(self, token_ids):
        """
        Decode token IDs back into text.
        """
        inv_vocab = {v: k for k, v in self.vocab.items()}
        return "".join([inv_vocab[token] for token in token_ids if token not in self.special_tokens.values()])

    def token_to_id(self, token):
        """
        Get the ID of a special token.
        """
        return self.special_tokens.get(token, self.vocab.get(token, -1))


In [71]:
import torch
import csv

class TranslationDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_path, tokenizer_src, tokenizer_tgt, src_lang="fr", tgt_lang="en", seq_len=50):
        super().__init__()
        self.seq_len = seq_len
        self.tokenizer_src = tokenizer_src
        self.tokenizer_tgt = tokenizer_tgt
        self.src_lang = src_lang
        self.tgt_lang = tgt_lang

        self.sos_token = torch.tensor([0], dtype=torch.int64)  # Define manually since custom tokenizer has no ID mapping
        self.eos_token = torch.tensor([1], dtype=torch.int64)
        self.pad_token = torch.tensor([2], dtype=torch.int64)

        # Load dataset
        self.dataset = []
        with open(dataset_path, newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                self.dataset.append({
                    "src_text": row[src_lang],
                    "tgt_text": row[tgt_lang]
                })

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        src_text = self.dataset[idx]['src_text']
        tgt_text = self.dataset[idx]['tgt_text']

        # Tokenize using custom BPE tokenizer
        enc_input_tokens = self.tokenizer_src.encode(src_text)
        dec_input_tokens = self.tokenizer_tgt.encode(tgt_text)

        # Ensure sequence length constraints
        enc_padding = self.seq_len - len(enc_input_tokens) - 2  # -2 for SOS and EOS
        dec_padding = self.seq_len - len(dec_input_tokens) - 1  # -1 for SOS

        if enc_padding < 0 or dec_padding < 0:
            raise ValueError("Sentence is too long for the specified sequence length.")

        # Prepare inputs with SOS/EOS/PAD
        encoder_input = torch.cat([
            self.sos_token,
            torch.tensor(enc_input_tokens, dtype=torch.int64),
            self.eos_token,
            torch.tensor([self.pad_token] * enc_padding, dtype=torch.int64),
        ])

        decoder_input = torch.cat([
            self.sos_token,
            torch.tensor(dec_input_tokens, dtype=torch.int64),
            torch.tensor([self.pad_token] * dec_padding, dtype=torch.int64),
        ])

        label = torch.cat([
            torch.tensor(dec_input_tokens, dtype=torch.int64),
            self.eos_token,
            torch.tensor([self.pad_token] * dec_padding, dtype=torch.int64),
        ])

        # Ensure shapes are correct
        assert encoder_input.size(0) == self.seq_len
        assert decoder_input.size(0) == self.seq_len
        assert label.size(0) == self.seq_len

        return {
            "encoder_input": encoder_input,
            "decoder_input": decoder_input,
            "encoder_mask": (encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int(),
            "decoder_mask": (decoder_input != self.pad_token).unsqueeze(0).int() & causal_mask(decoder_input.size(0)),
            "label": label,
            "src_text": src_text,
            "tgt_text": tgt_text,
        }

def causal_mask(size):
    """
    Create a triangular mask for the decoder. 
    This ensures that the model only attends to previous tokens and cannot "see the future" during training.
    """
    mask = torch.triu(torch.ones((1, size, size)), diagonal=1).type(torch.int)
    return mask == 0

In [72]:
from pathlib import Path

def get_config():
    return {
        "batch_size": 8,
        "num_epochs": 20,
        "lr": 10**-4,
        "seq": 350,
        "d_model": 512,
        "datasource": 'opus_books',
        "lang_src": "fr",
        "lang_tgt": "en",
        "model_folder": "weights",
        "model_basename": "tmodel_",
        "preload": "latest",
        "tokenizer_file": "tokenizer_{0}.json",
        "experiment_name": "runs/tmodel"
    }

def get_weights_file_path(config, epoch: str):
    model_folder = f"{config['datasource']}_{config['model_folder']}"
    model_filename = f"{config['model_basename']}{epoch}.pt"
    return str(Path('.') / model_folder / model_filename)

# Find the latest weights file in the weights folder
def latest_weights_file_path(config):
    model_folder = f"{config['datasource']}_{config['model_folder']}"
    model_filename = f"{config['model_basename']}*"
    weights_files = list(Path(model_folder).glob(model_filename))
    if len(weights_files) == 0:
        return None
    weights_files.sort()
    return str(weights_files[-1])

[UNK] → Unknown token = word or subword is not in the tokenizer's vocabulary.

In [73]:
import json
from pathlib import Path

def get_or_build_tokenizer(config, ds, lang):
    tokenizer_path = Path(config['tokenizer_file'].format(lang))

    if not tokenizer_path.exists():
        print(f"Training new tokenizer for {lang}...")
        
        # Initialize tokenizer with special tokens
        tokenizer = BPETokenizer(num_merges=100)
        tokenizer.special_tokens.update({
            "[UNK]": 3
        })

        # Train the tokenizer
        tokenizer.train(get_all_sentences(ds, lang))

        # Save tokenizer as JSON
        with open(tokenizer_path, "w", encoding="utf-8") as f:
            json.dump({
                "vocab": tokenizer.vocab,
                "merges": tokenizer.merges,
                "special_tokens": tokenizer.special_tokens
            }, f)
    else:
        print(f"Loading tokenizer from {tokenizer_path}...")
        
        # Load tokenizer from file
        with open(tokenizer_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        tokenizer = BPETokenizer()
        tokenizer.vocab = data["vocab"]
        tokenizer.merges = data["merges"]
        tokenizer.special_tokens = data["special_tokens"]

    return tokenizer